In [ ]:
# Step 1: Install dependencies
!pip install rtree folium

In [ ]:
from rtree import index
from math import radians, cos, sin, asin, sqrt
import folium

# Haversine distance function (km)
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r

# Sample places: id -> (name, category, (lon, lat))
places = {
    1: ("Coffee Shop A", "coffee", (77.59, 12.97)),
    2: ("Bookstore", "bookstore", (77.58, 12.95)),
    3: ("Coffee Shop B", "coffee", (77.60, 12.96)),
    4: ("Museum", "museum", (77.55, 12.92)),
    5: ("Coffee Shop C", "coffee", (77.62, 12.99)),  # Outside search area
    6: ("Gym", "gym", (77.61, 12.94)),
    7: ("Coffee Shop D", "coffee", (77.56, 12.93)),  # Outside search area
}

# Build R-tree index
idx = index.Index()
for place_id, (name, category, (x, y)) in places.items():
    idx.insert(place_id, (x, y, x, y))

# User inputs search bounding box: (min_lon, min_lat, max_lon, max_lat)
user_search_area = (77.57, 12.94, 77.61, 12.98)
user_lon = (user_search_area[0] + user_search_area[2]) / 2
user_lat = (user_search_area[1] + user_search_area[3]) / 2

# Search coffee shops inside area and calculate distances
def search_coffee_shops(index_obj, search_area, user_lon, user_lat):
    print(f"Searching coffee shops inside area: {search_area}")
    results = list(index_obj.intersection(search_area))

    coffee_shops_found = []
    for place_id in results:
        name, category, (lon, lat) = places[place_id]
        if category == "coffee":
            dist = haversine(user_lon, user_lat, lon, lat)
            coffee_shops_found.append((name, (lon, lat), dist))

    coffee_shops_found.sort(key=lambda x: x[2])

    if not coffee_shops_found:
        print("No coffee shops found in the search area.")
    else:
        print(f"Found {len(coffee_shops_found)} coffee shop(s):")
        for name, coord, dist in coffee_shops_found:
            print(f"- {name} at {coord}, Distance: {dist:.2f} km")
    return coffee_shops_found

coffee_shops = search_coffee_shops(idx, user_search_area, user_lon, user_lat)

# Create interactive map
m = folium.Map(location=[user_lat, user_lon], zoom_start=14)

# Draw search area rectangle
bounds = [(user_search_area[1], user_search_area[0]), (user_search_area[3], user_search_area[2])]
folium.Rectangle(bounds=bounds, color="blue", fill=True, fill_opacity=0.1, tooltip="Search Area").add_to(m)

# Mark user location
folium.Marker(
    location=[user_lat, user_lon],
    popup="User Search Center",
    icon=folium.Icon(color='green', icon='user')
).add_to(m)

# Mark coffee shops on map
for name, (lon, lat), dist in coffee_shops:
    folium.Marker(
        location=[lat, lon],
        popup=f"{name}<br>Distance: {dist:.2f} km",
        icon=folium.Icon(color='red', icon='coffee', prefix='fa')
    ).add_to(m)

# Show the map
m


Searching coffee shops inside area: (77.57, 12.94, 77.61, 12.98)
Found 2 coffee shop(s):
- Coffee Shop B at (77.6, 12.96), Distance: 1.08 km
- Coffee Shop A at (77.59, 12.97), Distance: 1.11 km
